In [12]:
# imports
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st
#from pandas_profiling import ProfileReport
#import missingno as msno

# extract and load data properly

## build up fct elements

In [15]:
csv_path = './data/'

In [16]:
file_names = [x for x in os.listdir(csv_path) if ".csv" in x]
file_names

['EdStatsData.csv',
 'EdStatsCountry-Series.csv',
 'EdStatsSeries.csv',
 'EdStatsCountry.csv',
 'EdStatsFootNote.csv']

In [18]:
file_names = [x.removesuffix('.csv') for x in file_names]
file_names = [x.removeprefix('EdStats') for x in file_names]
file_names

['Data', 'Country-Series', 'Series', 'Country', 'FootNote']

In [19]:
# construct dict data

dataframe_list = [f'DataFrame{i}' for i, item in enumerate(file_names, start=1)]
dataframe_list

['DataFrame1', 'DataFrame2', 'DataFrame3', 'DataFrame4', 'DataFrame5']

In [21]:
data = dict(zip(file_names, dataframe_list))
data

{'Data': 'DataFrame1',
 'Country-Series': 'DataFrame2',
 'Series': 'DataFrame3',
 'Country': 'DataFrame4',
 'FootNote': 'DataFrame5'}

## summarize in a function

In [45]:
import os
csv_path = './data/'

In [46]:
def get_data(csv_path):
    file_names = [x for x in os.listdir(csv_path) if ".csv" in x]
    # file_names_base = file_names
    key_names = [
        key_name.removesuffix('.csv').removeprefix('EdStats')
        for key_name in file_names
    ]

    data_dict = {}
    for k, f in zip(key_names, file_names):
        data_dict[k] = pd.read_csv(os.path.join(csv_path, f))

    return data_dict

In [47]:
data = get_data(csv_path)

# simple explo

## check dataset

In [153]:
data.keys()

dict_keys(['Data', 'Country-Series', 'Series', 'Country', 'FootNote'])

In [49]:
data['Data'].nunique()

Country Name        242
Country Code        242
Indicator Name     3665
Indicator Code     3665
1970              24595
                  ...  
2085               7335
2090               7150
2095               7044
2100               6914
Unnamed: 69           0
Length: 70, dtype: int64

## cardinality

In [50]:
country2 = data['Country'][['Country Code','Short Name','Income Group']]

In [51]:
# find distribution of different "Income Group"
country2.groupby(['Income Group']).count()

,Country Code,Short Name
Income Group,,
High income: OECD,31,31
High income: nonOECD,44,44
Low income,34,34
Lower middle income,50,50
Upper middle income,55,55


In [52]:
data['Data'].head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
data['Data'].shape

(886930, 70)

In [55]:
data['Data'].columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2020', '2025', '2030', '2035', '2040', '2045',
       '2050', '2055', '2060', '2065', '2070', '2075', '2080', '2085', '2090',
       '2095', '2100', 'Unnamed: 69'],
      dtype='object')

In [57]:
data['Data'].nunique()

Country Name        242
Country Code        242
Indicator Name     3665
Indicator Code     3665
1970              24595
                  ...  
2085               7335
2090               7150
2095               7044
2100               6914
Unnamed: 69           0
Length: 70, dtype: int64

### tests

In [61]:
maindata = data['Data'][['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']]
maindata

,Country Name,Country Code,Indicator Code
0,Arab World,ARB,UIS.NERA.2
1,Arab World,ARB,UIS.NERA.2.F
2,Arab World,ARB,UIS.NERA.2.GPI
3,Arab World,ARB,UIS.NERA.2.M
4,Arab World,ARB,SE.PRM.TENR
...,...,...,...
886925,Zimbabwe,ZWE,UIS.LP.AG15T24.M
886926,Zimbabwe,ZWE,SE.ADT.1524.LT.ZS
886927,Zimbabwe,ZWE,SE.ADT.1524.LT.FE.ZS
886928,Zimbabwe,ZWE,SE.ADT.1524.LT.FM.ZS


In [79]:
maindata.iloc[1,:]

Country Name        Arab World
Country Code               ARB
Indicator Code    UIS.NERA.2.F
Name: 1, dtype: object

In [85]:
maindata.duplicated().sum()

0

In [155]:
data['Data'].duplicated().sum()

0

In [161]:
#data[list(data.keys())[0]]
data['Data']

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886925,Zimbabwe,ZWE,"Youth illiterate population, 15-24 years, male...",UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886926,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, b...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886927,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, f...",SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886928,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, g...",SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
data['Data'].isnull().count()

Country Name      886930
Country Code      886930
Indicator Name    886930
Indicator Code    886930
1970              886930
                   ...  
2085              886930
2090              886930
2095              886930
2100              886930
Unnamed: 69       886930
Length: 70, dtype: int64

In [116]:
data['Data'].count()

Country Name      886930
Country Code      886930
Indicator Name    886930
Indicator Code    886930
1970               72288
                   ...  
2085               51436
2090               51436
2095               51436
2100               51436
Unnamed: 69            0
Length: 70, dtype: int64

In [123]:
minor = data['Data'][['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2020', '2025', '2030', '2035', '2040', '2045',
       '2050', '2055', '2060', '2065', '2070', '2075', '2080', '2085', '2090',
       '2095', '2100']]

In [133]:
preds = data['Data'][['Country Name', 'Country Code', 'Indicator Code','2020', '2025', '2030', '2035', '2040', '2045',
       '2050', '2055', '2060', '2065', '2070', '2075', '2080', '2085', '2090',
       '2095', '2100']]

In [144]:
preds

,Country Name,Country Code,Indicator Code,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,Arab World,ARB,UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886925,Zimbabwe,ZWE,UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886926,Zimbabwe,ZWE,SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886927,Zimbabwe,ZWE,SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886928,Zimbabwe,ZWE,SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
preds.dropna().count()

Country Name      51436
Country Code      51436
Indicator Code    51436
2020              51436
2025              51436
2030              51436
2035              51436
2040              51436
2045              51436
2050              51436
2055              51436
2060              51436
2065              51436
2070              51436
2075              51436
2080              51436
2085              51436
2090              51436
2095              51436
2100              51436
dtype: int64

In [140]:
# variables ayant des valeurs manquantes
preds_na = preds.isnull().sum()
preds_na[preds_na>0]

2020    835494
2025    835494
2030    835494
2035    835494
2040    835494
2045    835494
2050    835494
2055    835494
2060    835494
2065    835494
2070    835494
2075    835494
2080    835494
2085    835494
2090    835494
2095    835494
2100    835494
dtype: int64

## TODO : take 1 value per country per indicator

## explo work

In [168]:
# select all rows with column 'Indicator Code' with value 'UIS.XGDP.56.FSGOV' 
#(Government expenditure on education as % of GDP (%))

XGDP_Data = data['Data'].loc[data['Data']['Indicator Code'] == 'UIS.XGDP.56.FSGOV']
XGDP_Data

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
1266,Arab World,ARB,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4931,East Asia & Pacific,EAS,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8596,East Asia & Pacific (excluding high income),EAP,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12261,Euro area,EMU,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15926,Europe & Central Asia,ECS,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869871,Virgin Islands (U.S.),VIR,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873536,West Bank and Gaza,PSE,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877201,"Yemen, Rep.",YEM,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880866,Zambia,ZMB,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
# check if all country codes are different

XGDP_Data['Country Code'].duplicated().sum()

0

## fct build

In [174]:
## fonction de transformation

def select_column_val(df, col, val):
    df_select = df.loc[df[col] == val]
    return df_select


In [175]:
data_XGDP = select_column_val(data['Data'],'Indicator Code','UIS.XGDP.56.FSGOV')
data_XGDP

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
1266,Arab World,ARB,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4931,East Asia & Pacific,EAS,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8596,East Asia & Pacific (excluding high income),EAP,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12261,Euro area,EMU,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15926,Europe & Central Asia,ECS,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869871,Virgin Islands (U.S.),VIR,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873536,West Bank and Gaza,PSE,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877201,"Yemen, Rep.",YEM,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880866,Zambia,ZMB,Government expenditure on tertiary education a...,UIS.XGDP.56.FSGOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
#create df with only 'years' columns
data_XGDP_years = data_XGDP.loc[:, ~data_XGDP.columns.isin(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])]
data_XGDP_years.shape

(242, 66)

In [200]:
#calcule % of missing values within 'years' columns
percent_missing = data_XGDP_years.isnull().sum() * 100 / len(data_XGDP_years)

#create df listing df column names with missing values % 
missing_values_df = pd.DataFrame({'column_name':data_XGDP_years.columns,
                                  'percent_missing': percent_missing
                                 })
#
missing_values_df[missing_values_df.percent_missing !=100]

,column_name,percent_missing
1998,1998,76.033058
1999,1999,70.661157
2000,2000,71.074380
2001,2001,67.768595
2002,2002,63.223140
2003,2003,71.487603
2004,2004,61.570248
2005,2005,64.462810
2006,2006,63.636364
2007,2007,63.636364
